In [2]:
import sys
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install -U textblob
!{sys.executable} -m pip install preprocessor
!{sys.executable} -m pip install tweet-preprocessor
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install argparse
!{sys.executable} -m pip install -U nltk



Requirement already up-to-date: textblob in /Users/User/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages (0.15.3)
     |████████████████████████████████| 1.5MB 4.3MB/s eta 0:00:01
  Stored in directory: /Users/User/Library/Caches/pip/wheels/54/40/b7/c56ad418e6cd4d9e1e594b5e138d1ca6eec11a6ee3d464e5bb
Successfully built nltk
  Found existing installation: nltk 3.4.1
    Uninstalling nltk-3.4.1:
      Successfully uninstalled nltk-3.4.1


In [3]:
import sys
import tweepy
import os
import sklearn
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import argparse
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords, wordnet

In [4]:
consumer_key = 'LvANfv2UjDSQV9UfmbBnYWOfc'
consumer_secret = 'xlwH6LWPYTpOMexSwb8q8FHcYhvqRPzwLDqKkYes6c9tcT9rZJ'
access_key= '1122632146382835712-lTVNdM6snnpoN0aF4oCUqpRAO5l7rv'
access_secret = 'wo5UI1Gkcb7BNzZPVW5fV8J7gxK6sfGtH1YdWUJRk2B2y'

In [5]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [6]:
#columns of the csv file
COLS = ['id', 'label', 'clean_text', 'screen_name']

In [7]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [8]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [9]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


In [10]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [11]:
def clean_tweets(tweet):
    
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'RT *[^:]*:','', tweet);
    #tweet = re.sub(r'[\.@[a-zA-Z]*:]+','', tweet);
    tweet = re.sub(r':', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#remove url
    tweet = re.sub(r"http\S+", "", tweet)
#remove more symbols
    tweet = tweet.replace("-", "")
    tweet = tweet.replace("_", "")
    tweet = tweet.replace("*", "")
    tweet = tweet.replace(".", "")
#filter using NLTK library append it to a string
    filtered_tweet = []
    word_tokens = word_tokenize(tweet)
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in emoticons and w not in string.punctuation:
            if w.isdigit():
                continue
            if w[0] == ".":
                continue
            if '9' in w:
                w = w.replace("9", "")
                w = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', w)
            
            w = w.lower()
            
            if len(w) == 1 and w not in ['a', 'e', 'i','o','u']:
                continue
            elif len(w) == 2 and w[0] in string.punctuation:
                continue
            filtered_tweet.append(w.lower())
    return ' '.join(filtered_tweet)
    #print(word_tokens)

In [12]:
contractions_dict = { 
"ain't": "have not",
"aren't": "are not",
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he has",
"how'd": "how did",
"how'll": "how will",
"how's": "how are",
"i'd": "I would",
"i'll": "I will",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"o'clock": "of the clock",
"oughtn't": "ought not",
"she'd": "she would",
"she'll": "she will",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there would",
"there's": "there has",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"y'all": "you all",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"you've": "you have"
}


In [13]:
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()), flags = re.IGNORECASE)
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0).lower()]
    return contractions_re.sub(replace, s)

In [14]:
def write_tweets(handle, file):
    #If the file exists, then read the existing data from the CSV file.
    #if os.path.exists(file):
        #df = pd.read_csv(file, header=0)
   # else:
        #df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    global df
    for page in tweepy.Cursor(api.user_timeline, id=handle,
                              count=200, include_rts=True, tweet_mode = 'extended').pages():
        for status in page:
            new_entry = []
            status = status._json
            
            if "retweeted_status" in status:
                status = status["retweeted_status"]
            
            original_tweet = status['full_text']
            
            if status['id'] in df['id'].values:
                continue
            
            tweet = status['full_text'].replace("&amp;", "and") #replace with full text
            tweet = tweet.replace("’", "'")
            tweet = tweet.replace("#", "9")
            try:
                tweet = expand_contractions(tweet)
            except:
                print("contraction error")

            p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY)
            clean_text = p.clean(tweet)
            filtered_tweet=clean_tweets(clean_text)

            
            filtered_tweet = filtered_tweet.strip()
            if len(filtered_tweet.split()) < 2:
                continue

            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment     
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
            
            new_entry += [status['id'],filtered_tweet, "rep", status['user']['screen_name']]
        
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            #print(df.shape)
            csvFile = open(file, 'a+' ,encoding='utf-8')
            single_tweet_df.to_csv(file, mode='a+', columns=COLS, header = False, index=False, encoding="utf-8")
            print("success")
            
            #csvFile = open(file, 'a+' ,encoding='utf-8')
            #df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

In [39]:
#newDf = pd.DataFrame(columns=COLS)
df.to_csv("../data/new_party_data.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")

/Users/User/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [64]:
testDf = pd.read_csv("/Users/User/221project/data/party_data2.csv")
testDf.drop_duplicates('id', inplace = True)
testDf.shape

(624, 8)

In [58]:
df = pd.read_csv("/Users/User/221project/data/party_data2.csv")
df.drop_duplicates('id', inplace = True)
df.shape

(118616, 4)

In [61]:
tweets = "/Users/User/221project/data/new_party_data.csv"


'''
senDems = "SenateDems"
write_tweets(senDems, tweets)
houseDems = "HouseDemocrats"
write_tweets(houseDems, tweets)
senSan = "SenSanders"
write_tweets(senSan, tweets)
aoc = "AOC"
write_tweets(aoc, tweets)
spel = "SpeakerPelosi"
write_tweets(spel, tweets)
schum = "SenSchumer"
write_tweets(schum, tweets)
khar = "KamalaHarris"
write_tweets(khar, tweets)
senWar = "SenWarren"
write_tweets(senWar, tweets)
corB = "CoryBooker"
write_tweets(corB, tweets)
aS = "RepAdamSchiff"
write_tweets(aS, tweets)
jb = "JoeBiden"
write_tweets(jb, tweets)
hrc = "HillaryClinton"
write_tweets(hrc, tweets)
ilhan = "IlhanMN"
write_tweets(ilhan, tweets)
ppact = "PPact"
write_tweets(ppact, tweets)
gnew = "GavinNewsom"
write_tweets(gnew, tweets)
ted = "RepTedLieu"
write_tweets(ted, tweets)
eric = "ericswalwell"
write_tweets(eric, tweets)
kgil = "SenGillibrand"
write_tweets(kgil, tweets)


'''
rep_handle1 = "SenateGOP"
rep_handle2 = "HouseGOP"
rep_handle3 = "realDonaldTrump"
rep_handle4 = "senatemajldr"
rep_handle5 = "SpeakerRyan"
rep_handle6 = "Heritage"
rep_handle7 = "GOPChairwoman"
rep_handle8 = "RepMattGaetz"
rep_handle9 = "marcorubio"
rep_handle10 = "WarrenDavidson"
rep_handle11 = "GOPLeader"
rep_handle12= "DLoesch"
rep_handle13 = "SteveScalise"
rep_handle14 = "March_For_Life"
rep_handle15 = "charliekirk11"
rep_handle16 = "SenatorTimScott"
rep_handle17 = "SenMcSallyAZ"
rep_handle18 = "TedCruz"
rep_handle19 = "KellyannePolls"

write_tweets(rep_handle1, tweets)
write_tweets(rep_handle2, tweets)
write_tweets(rep_handle3, tweets)
write_tweets(rep_handle4, tweets)
write_tweets(rep_handle5, tweets)
write_tweets(rep_handle6, tweets)
write_tweets(rep_handle7, tweets)
write_tweets(rep_handle8, tweets)
write_tweets(rep_handle9, tweets)
write_tweets(rep_handle10, tweets)
write_tweets(rep_handle11, tweets)
write_tweets(rep_handle12, tweets)
write_tweets(rep_handle13, tweets)
write_tweets(rep_handle14, tweets)
write_tweets(rep_handle15, tweets)
write_tweets(rep_handle16, tweets)
write_tweets(rep_handle17, tweets)
write_tweets(rep_handle18, tweets)
write_tweets(rep_handle19, tweets)


success


In [66]:
write_tweets("NYCMayor", tweets)
#write_tweets("PeteButtigieg", tweets)
#write_tweets("SenFeinstein", tweets)
#write_tweets("JulianCastro", tweets)

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [68]:
write_tweets("TomiLahren", tweets)
write_tweets("DanCrenshawTX", tweets)
write_tweets("RepMarkMeadows", tweets)
write_tweets("Jim_Jordan", tweets)

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [84]:
df = pd.read_csv("/Users/User/221project/data/new_party_data.csv")
df.shape

(1249, 4)

In [70]:
df = df[df['screen_name'] != "NYCMayor"]

In [71]:
df.shape

(137228, 4)

In [79]:
df.to_csv("../data/new_party_data_correct.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")

In [95]:
df = pd.read_csv("/Users/User/221project/data/actual_new_party_data.csv")
df.shape
df

,id,clean_text,label,screen_name
0,1.040000e+18,you are never alone if you or someone you know...,dem,KamalaHarris
1,1.060000e+18,these targeted acts of terror are despicable c...,dem,CoryBooker
2,8.600000e+17,with or without preexisting conditions people ...,rep,AmericaNewsroom
3,9.390000e+17,riding on air force one with our outstanding p...,rep,RepMattGaetz
4,9.380000e+17,evacuation alert is in effect in parts of ca33...,dem,RepTedLieu
5,7.930000e+17,media hope you will ask clinton team what they...,rep,KellyannePolls
6,9.820000e+17,my final thoughts,rep,TomiLahren
7,9.570000e+17,tonight do not miss my interview with laura in...,rep,RepMattGaetz
8,1.010000e+18,another attack against our brothers and sister...,dem,sethmoulton
9,9.250000e+17,are implying that there should be a religious ...,rep,SenateGOP


In [96]:
from sklearn.utils import shuffle
df = shuffle(df)

In [102]:
df
df.shape

(137228, 4)

In [100]:
df[['label','clean_text']] = df[['clean_text','label']].where(df['clean_text'] == "rep", df[['label','clean_text']].values)
df.shape

(137228, 4)

In [101]:
df[['label','clean_text']] = df[['clean_text','label']].where(df['clean_text'] == "dem", df[['label','clean_text']].values)

In [103]:
df
df.to_csv("../data/another_try.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")

In [106]:
newDf1 = df[df['clean_text'] == "dem"]

In [107]:
newDf1

,id,clean_text,label,screen_name


In [110]:
testTweets = df.sample(n=300)

In [112]:
testTweets.to_csv("../data/test_tweets.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")

In [118]:
testDF = pd.read_csv("/Users/User/221project/data/test_tweets.csv")

In [119]:
testDF.loc[testDF['label'] == 'dem', 'label'] = 0

In [121]:
testDF.loc[testDF['label'] == 'rep', 'label'] = 1

In [123]:
testDF = testDF[['clean_text', 'label','screen_name','id']]

In [125]:
testDF
testDF.to_csv("../data/test_tweets2.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")

In [60]:
df= pd.read_csv("/Users/User/221project/data/new_data.csv")
oldDf = pd.read_csv("/Users/User/221project/data/party_data2.csv")
df = df.append(oldDf)
df.drop_duplicates('clean_text', inplace=True)
df.shape

(128101, 10)

In [37]:
duplicateRowsDF = df[df.duplicated(['id'])]
df.sh

(135979, 4)

In [33]:
df.to_csv("../data/umm.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")

In [39]:
duplicateRowsDF.to_csv("../data/umm2.csv", mode='a+', columns=COLS,index=False, encoding="utf-8")